In [5]:
!conda clean -tipsy
#import sys
#!conda install --yes --prefix {sys.prefix} keras
#!conda install --yes --prefix {sys.prefix} os
#!conda install --yes --prefix {sys.prefix} scipy
#!conda install --yes --prefix {sys.prefix} matplotlib
#!conda install --yes --prefix {sys.prefix} pandas
#!conda install --yes --prefix {sys.prefix} tensorflow


Cache location: 
There are no tarballs to remove
Cache location: 
There are no unused packages to remove


    Use 'conda build purge-all' to remove source cache files.


In [6]:
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
from keras.utils import np_utils

import numpy as np

import os

import scipy.io
import scipy.misc

import matplotlib.pyplot as plt

import tensorflow as tf

import pandas as pd

from sklearn.model_selection import train_test_split

from PIL import Image

#import cv2

#from datetime import datetime

Using TensorFlow backend.
C:\Users\grave\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\grave\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\grave\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\grave\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [7]:
num_classes = 101
weight_decay = 0.0005
x_shape = [50,50,3]

In [8]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 50, 50, 64)        0         
________________________________________________

In [9]:
def imread(path):
    img = scipy.misc.imread(path).astype(np.float)
    if len(img.shape) == 2:
        img = np.transpose(np.array([img, img, img]), (2, 0, 1))
    return img
    
path = "C:/Users/grave/Desktop/Deep Learning Neural Networks/Project/101_ObjectCategories/train"
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

categories = sorted(os.listdir(path))
ncategories = len(categories)
imgs = []
labels = []
# LOAD ALL IMAGES 
k = 0
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            #print('fullpath = ', fullpath)
            
            ###Image Attempt###
            img = np.array(Image.open(fullpath).resize((50,50)))

            # Grayscale images will not have 3 channels, only 1. Stack 2 more
            # for RGB conversion
            if len(img.shape) < 3:
              img = np.stack((img,img,img), axis=2)
            #print('img size = ',img.size)

            ###CV2 ATTEMPT###
            #img = cv2.imread(fullpath)
            #print('img size = ',img.size)
            #img = cv2.resize(img,[32,32,3])

            ###ORIGINAL###
            #img = scipy.misc.imresize(imread(fullpath), [32,32, 3])
            img = img.astype('float32')

            

            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            imgs.append(img) # NORMALIZE IMAGE 
            label_curr = i
            labels.append(label_curr)
            k = k + 1
            print(k, ' out of 9000')
        #iter = (iter+1)%10;
print ("Num imgs: %d" % (len(imgs)))
print ("Num labels: %d" % (len(labels)) )
print (ncategories)

[101] CATEGORIES ARE IN 
 C:/Users/grave/Desktop/Deep Learning Neural Networks/Project/101_ObjectCategories/train
1  out of 9000
2  out of 9000
3  out of 9000
4  out of 9000
5  out of 9000
6  out of 9000
7  out of 9000
8  out of 9000
9  out of 9000
10  out of 9000
11  out of 9000
12  out of 9000
13  out of 9000
14  out of 9000
15  out of 9000
16  out of 9000
17  out of 9000
18  out of 9000
19  out of 9000
20  out of 9000
21  out of 9000
22  out of 9000
23  out of 9000
24  out of 9000
25  out of 9000
26  out of 9000
27  out of 9000
28  out of 9000
29  out of 9000
30  out of 9000
31  out of 9000
32  out of 9000
33  out of 9000
34  out of 9000
35  out of 9000
36  out of 9000
37  out of 9000
38  out of 9000
39  out of 9000
40  out of 9000
41  out of 9000
42  out of 9000
43  out of 9000
44  out of 9000
45  out of 9000
46  out of 9000
47  out of 9000
48  out of 9000
49  out of 9000
50  out of 9000
51  out of 9000
52  out of 9000
53  out of 9000
54  out of 9000
55  out of 9000
56  out of 9000

487  out of 9000
488  out of 9000
489  out of 9000
490  out of 9000
491  out of 9000
492  out of 9000
493  out of 9000
494  out of 9000
495  out of 9000
496  out of 9000
497  out of 9000
498  out of 9000
499  out of 9000
500  out of 9000
501  out of 9000
502  out of 9000
503  out of 9000
504  out of 9000
505  out of 9000
506  out of 9000
507  out of 9000
508  out of 9000
509  out of 9000
510  out of 9000
511  out of 9000
512  out of 9000
513  out of 9000
514  out of 9000
515  out of 9000
516  out of 9000
517  out of 9000
518  out of 9000
519  out of 9000
520  out of 9000
521  out of 9000
522  out of 9000
523  out of 9000
524  out of 9000
525  out of 9000
526  out of 9000
527  out of 9000
528  out of 9000
529  out of 9000
530  out of 9000
531  out of 9000
532  out of 9000
533  out of 9000
534  out of 9000
535  out of 9000
536  out of 9000
537  out of 9000
538  out of 9000
539  out of 9000
540  out of 9000
541  out of 9000
542  out of 9000
543  out of 9000
544  out of 9000
545  out of 90

984  out of 9000
985  out of 9000
986  out of 9000
987  out of 9000
988  out of 9000
989  out of 9000
990  out of 9000
991  out of 9000
992  out of 9000
993  out of 9000
994  out of 9000
995  out of 9000
996  out of 9000
997  out of 9000
998  out of 9000
999  out of 9000
1000  out of 9000
1001  out of 9000
1002  out of 9000
1003  out of 9000
1004  out of 9000
1005  out of 9000
1006  out of 9000
1007  out of 9000
1008  out of 9000
1009  out of 9000
1010  out of 9000
1011  out of 9000
1012  out of 9000
1013  out of 9000
1014  out of 9000
1015  out of 9000
1016  out of 9000
1017  out of 9000
1018  out of 9000
1019  out of 9000
1020  out of 9000
1021  out of 9000
1022  out of 9000
1023  out of 9000
1024  out of 9000
1025  out of 9000
1026  out of 9000
1027  out of 9000
1028  out of 9000
1029  out of 9000
1030  out of 9000
1031  out of 9000
1032  out of 9000
1033  out of 9000
1034  out of 9000
1035  out of 9000
1036  out of 9000
1037  out of 9000
1038  out of 9000
1039  out of 9000
1040  ou

1448  out of 9000
1449  out of 9000
1450  out of 9000
1451  out of 9000
1452  out of 9000
1453  out of 9000
1454  out of 9000
1455  out of 9000
1456  out of 9000
1457  out of 9000
1458  out of 9000
1459  out of 9000
1460  out of 9000
1461  out of 9000
1462  out of 9000
1463  out of 9000
1464  out of 9000
1465  out of 9000
1466  out of 9000
1467  out of 9000
1468  out of 9000
1469  out of 9000
1470  out of 9000
1471  out of 9000
1472  out of 9000
1473  out of 9000
1474  out of 9000
1475  out of 9000
1476  out of 9000
1477  out of 9000
1478  out of 9000
1479  out of 9000
1480  out of 9000
1481  out of 9000
1482  out of 9000
1483  out of 9000
1484  out of 9000
1485  out of 9000
1486  out of 9000
1487  out of 9000
1488  out of 9000
1489  out of 9000
1490  out of 9000
1491  out of 9000
1492  out of 9000
1493  out of 9000
1494  out of 9000
1495  out of 9000
1496  out of 9000
1497  out of 9000
1498  out of 9000
1499  out of 9000
1500  out of 9000
1501  out of 9000
1502  out of 9000
1503  out 

1905  out of 9000
1906  out of 9000
1907  out of 9000
1908  out of 9000
1909  out of 9000
1910  out of 9000
1911  out of 9000
1912  out of 9000
1913  out of 9000
1914  out of 9000
1915  out of 9000
1916  out of 9000
1917  out of 9000
1918  out of 9000
1919  out of 9000
1920  out of 9000
1921  out of 9000
1922  out of 9000
1923  out of 9000
1924  out of 9000
1925  out of 9000
1926  out of 9000
1927  out of 9000
1928  out of 9000
1929  out of 9000
1930  out of 9000
1931  out of 9000
1932  out of 9000
1933  out of 9000
1934  out of 9000
1935  out of 9000
1936  out of 9000
1937  out of 9000
1938  out of 9000
1939  out of 9000
1940  out of 9000
1941  out of 9000
1942  out of 9000
1943  out of 9000
1944  out of 9000
1945  out of 9000
1946  out of 9000
1947  out of 9000
1948  out of 9000
1949  out of 9000
1950  out of 9000
1951  out of 9000
1952  out of 9000
1953  out of 9000
1954  out of 9000
1955  out of 9000
1956  out of 9000
1957  out of 9000
1958  out of 9000
1959  out of 9000
1960  out 

...

6532  out of 9000
6533  out of 9000
6534  out of 9000
6535  out of 9000
6536  out of 9000
6537  out of 9000
6538  out of 9000
6539  out of 9000
6540  out of 9000
6541  out of 9000
6542  out of 9000
6543  out of 9000
6544  out of 9000
6545  out of 9000
6546  out of 9000
6547  out of 9000
6548  out of 9000
6549  out of 9000
6550  out of 9000
6551  out of 9000
6552  out of 9000
6553  out of 9000
6554  out of 9000
6555  out of 9000
6556  out of 9000
6557  out of 9000
6558  out of 9000
6559  out of 9000
6560  out of 9000
6561  out of 9000
6562  out of 9000
6563  out of 9000
6564  out of 9000
6565  out of 9000
6566  out of 9000
6567  out of 9000
6568  out of 9000
6569  out of 9000
6570  out of 9000
6571  out of 9000
6572  out of 9000
6573  out of 9000
6574  out of 9000
6575  out of 9000
6576  out of 9000
6577  out of 9000
6578  out of 9000
6579  out of 9000
6580  out of 9000
6581  out of 9000
6582  out of 9000
6583  out of 9000
6584  out of 9000
6585  out of 9000
6586  out of 9000
6587  out 

6999  out of 9000
7000  out of 9000
7001  out of 9000
7002  out of 9000
7003  out of 9000
7004  out of 9000
7005  out of 9000
7006  out of 9000
7007  out of 9000
7008  out of 9000
7009  out of 9000
7010  out of 9000
7011  out of 9000
7012  out of 9000
7013  out of 9000
7014  out of 9000
7015  out of 9000
7016  out of 9000
7017  out of 9000
7018  out of 9000
7019  out of 9000
7020  out of 9000
7021  out of 9000
7022  out of 9000
7023  out of 9000
7024  out of 9000
7025  out of 9000
7026  out of 9000
7027  out of 9000
7028  out of 9000
7029  out of 9000
7030  out of 9000
7031  out of 9000
7032  out of 9000
7033  out of 9000
7034  out of 9000
7035  out of 9000
7036  out of 9000
7037  out of 9000
7038  out of 9000
7039  out of 9000
7040  out of 9000
7041  out of 9000
7042  out of 9000
7043  out of 9000
7044  out of 9000
7045  out of 9000
7046  out of 9000
7047  out of 9000
7048  out of 9000
7049  out of 9000
7050  out of 9000
7051  out of 9000
7052  out of 9000
7053  out of 9000
7054  out 

7456  out of 9000
7457  out of 9000
7458  out of 9000
7459  out of 9000
7460  out of 9000
7461  out of 9000
7462  out of 9000
7463  out of 9000
7464  out of 9000
7465  out of 9000
7466  out of 9000
7467  out of 9000
7468  out of 9000
7469  out of 9000
7470  out of 9000
7471  out of 9000
7472  out of 9000
7473  out of 9000
7474  out of 9000
7475  out of 9000
7476  out of 9000
7477  out of 9000
7478  out of 9000
7479  out of 9000
7480  out of 9000
7481  out of 9000
7482  out of 9000
7483  out of 9000
7484  out of 9000
7485  out of 9000
7486  out of 9000
7487  out of 9000
7488  out of 9000
7489  out of 9000
7490  out of 9000
7491  out of 9000
7492  out of 9000
7493  out of 9000
7494  out of 9000
7495  out of 9000
7496  out of 9000
7497  out of 9000
7498  out of 9000
7499  out of 9000
7500  out of 9000
7501  out of 9000
7502  out of 9000
7503  out of 9000
7504  out of 9000
7505  out of 9000
7506  out of 9000
7507  out of 9000
7508  out of 9000
7509  out of 9000
7510  out of 9000
7511  out 

7913  out of 9000
7914  out of 9000
7915  out of 9000
7916  out of 9000
7917  out of 9000
7918  out of 9000
7919  out of 9000
7920  out of 9000
7921  out of 9000
7922  out of 9000
7923  out of 9000
7924  out of 9000
7925  out of 9000
7926  out of 9000
7927  out of 9000
7928  out of 9000
7929  out of 9000
7930  out of 9000
7931  out of 9000
7932  out of 9000
7933  out of 9000
7934  out of 9000
7935  out of 9000
7936  out of 9000
7937  out of 9000
7938  out of 9000
7939  out of 9000
7940  out of 9000
7941  out of 9000
7942  out of 9000
7943  out of 9000
7944  out of 9000
7945  out of 9000
7946  out of 9000
7947  out of 9000
7948  out of 9000
7949  out of 9000
7950  out of 9000
7951  out of 9000
7952  out of 9000
7953  out of 9000
7954  out of 9000
7955  out of 9000
7956  out of 9000
7957  out of 9000
7958  out of 9000
7959  out of 9000
7960  out of 9000
7961  out of 9000
7962  out of 9000
7963  out of 9000
7964  out of 9000
7965  out of 9000
7966  out of 9000
7967  out of 9000
7968  out 

8371  out of 9000
8372  out of 9000
8373  out of 9000
8374  out of 9000
8375  out of 9000
8376  out of 9000
8377  out of 9000
8378  out of 9000
8379  out of 9000
8380  out of 9000
8381  out of 9000
8382  out of 9000
8383  out of 9000
8384  out of 9000
8385  out of 9000
8386  out of 9000
8387  out of 9000
8388  out of 9000
8389  out of 9000
8390  out of 9000
8391  out of 9000
8392  out of 9000
8393  out of 9000
8394  out of 9000
8395  out of 9000
8396  out of 9000
8397  out of 9000
8398  out of 9000
8399  out of 9000
8400  out of 9000
8401  out of 9000
8402  out of 9000
8403  out of 9000
8404  out of 9000
8405  out of 9000
8406  out of 9000
8407  out of 9000
8408  out of 9000
8409  out of 9000
8410  out of 9000
8411  out of 9000
8412  out of 9000
8413  out of 9000
8414  out of 9000
8415  out of 9000
8416  out of 9000
8417  out of 9000
8418  out of 9000
8419  out of 9000
8420  out of 9000
8421  out of 9000
8422  out of 9000
8423  out of 9000
8424  out of 9000
8425  out of 9000
8426  out 

In [10]:
seed = 7
np.random.seed(seed)
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size = 0.1)
x_train = np.stack(x_train, axis=0)
y_train = np.stack(y_train, axis=0)
x_test = np.stack(x_test, axis=0)
y_test = np.stack(y_test, axis=0)
print ("Num train_imgs: %d" % (len(x_train)))
print ("Num test_imgs: %d" % (len(x_test)))

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#num_classes= y_test.shape[1]

Num train_imgs: 7809
Num test_imgs: 868


In [11]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(7809, 50, 50, 3)
(7809, 101)
(868, 50, 50, 3)
(868, 101)


In [12]:
batch_size = 64
maxepoches = 25
learning_rate = 0.01
lr_decay = 1e-6
lr_drop = 20
momentum_val = 0.9

In [13]:
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)


#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [14]:
#now = datetime.now()

#current_time = now.strftime("%H:%M:%S")
#print("Current Time =", current_time)
datagen.fit(x_train)

#optimization details
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=momentum_val, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

# training process in a for loop with learning rate drop every 25 epoches.
hist = model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=maxepoches,
                    validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
model.save_weights('caltech101-vgg.h5')

#now = datetime.now()

#current_time = now.strftime("%H:%M:%S")
#print("Current Time =", current_time)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25


KeyboardInterrupt: 

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Training Accuracy: %.2f%%" % (hist.history['accuracy'][maxepoches - 1]*100))
print("Testing Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.legend(['Train','Test'])
plt.title('VGG16 Loss')
plt.savefig("VGG16_loss.png",dpi=300,format="png")
plt.figure()
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.legend(['Train','Test'])
plt.title('VGG16 Accuracy')
plt.savefig("VGG16_accuracy.png",dpi=300,format="png")
model_json = model.to_json()
with open("VGG16_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("VGG16_model.h5")
print("Saved model to disk")